## Linear Analyses
---

There are a range of linear metrics available for analysing time series data, particularly when the captured behavioural time series is continuous (rather than discrete) and includes highly regular, periodic, and stationary dynamics. 

These linear metrics include:
- Capturing amplitude or period
- Spectral analysis and cross-spectral coherence
- Auto and cross-correlation
- Relative phase analysis

#### Let's Practice Running Some Linear Analyses

Before we begin, we need to import and load various packages and utilities. These will allow us to import the data, manipulate it, run MdRQA, and create visualisations to explore our results.

The code below will do all the setup for you. Simply click the "play" button on the left to run the code, and we'll be ready to start our analyses. 

In [1]:
import os
import pandas as pd
from utils import filter_data, interpolate_missing_data
from utils.corr_utils import auto_correlation, cross_correlation, plot_autocorrelation, plot_crosscorrelation
from utils.period_amplitude_utils import period, amplitude, plot_period, plot_amplitude
from utils.coherence_utils import cross_spectral_coherence, plot_coherence, plot_spectral_analysis
from utils.relative_phase_utils import irp, drp, plot_irp, plot_drp, plot_rp_distribution, return_plot

Now that all the utilities are loaded, let's select some real data to use — in this case, positional data from two people [swinging pendulums](data/linearAnalyses/pendulums.txt).

Just click the "play" button below to load in and clean the data.

In [2]:
# Load data from csv file
data = pd.read_csv(os.path.join('data/linearAnalyses', 'pendulums.txt'), sep='\t', header=None)

# Convert all columns to numeric type to allow interpolation
data = data.apply(pd.to_numeric, errors='coerce')

# Interpolate any missing data that might be present in the file
data = interpolate_missing_data(data)

# Apply a filter to the data
data = filter_data(data)

# Normalise the data by using a z-score
data = (data - data.mean()) / data.std()


##### Mean Period and Amplitude

Let's start by calculating the mean period and amplitude of each pendulum. To do this, you'll need to set the sample rate, as well as the minimum peak distance (as a fraction of the sample rate) and the minimum peak height (as a fraction of the maximum value). For the current data these have been set to:
- Sample rate = 120
- Minimum peak distance - 0.3
- Minimum peak height - 0.1

Let's calculate the mean period for each person:

In [ ]:
# Calculates mean and SD period
## participant 1
meanPeriod, sdPeriod, peaks, pkLocs = period(data.iloc[:, 0], samplerate=120, DistFLT=0.3, AmpFLT=0.1)
print(f'Participant 1: \nMean period: {meanPeriod}')
print(f'Standard deviation period: {sdPeriod}')
plot_period(data.iloc[:, 0], 120, peaks, pkLocs)

## participant 2
meanPeriod, sdPeriod, peaks, pkLocs = period(data.iloc[:, 1], samplerate=120, DistFLT=0.3, AmpFLT=0.1)
print(f'Participant 2: \nMean period: {meanPeriod}')
print(f'Standard deviation period: {sdPeriod}')
plot_period(data.iloc[:, 1], 120, peaks, pkLocs)

##### Interpreting the Results

If your code ran successfully, you should see that each participant shows a similar mean and standard deviation of period (M = 0.4, SD = 0.01). You should also see a graph for each participant with the peaks highlighted.  

Now let's calculate the mean amplitude:

In [ ]:
# Calculates mean and SD amplitude
## participant 1
meanAmp, sdAmp, peaks, pkLocs, valleys, vLocs = amplitude(data.iloc[:, 0], samplerate=120, DistFLT=0.3, AmpFLT=0.1)
print(f'Participant 1: \nMean amplitude: {meanAmp}')
print(f'Standard deviation amplitude: {sdAmp}')
plot_amplitude(data.iloc[:, 0], 120, peaks, pkLocs, valleys, vLocs)

## participant 2
meanAmp, sdAmp, peaks, pkLocs, valleys, vLocs = amplitude(data.iloc[:, 1], samplerate=120, DistFLT=0.3, AmpFLT=0.1)
print(f'Participant 2: \nMean amplitude: {meanAmp}')
print(f'Standard deviation amplitude: {sdAmp}')
plot_amplitude(data.iloc[:, 1], 120, peaks, pkLocs, valleys, vLocs)

##### Interpreting the Results

If your code ran successfully, you should see that each participant shows a similar mean and standard deviation of amplitude (M = 1.4, SD = 0.1). You should also see a graph for each participant with the peaks and valleys highlighted.  

##### Spectral Analysis and Cross Spectral Coherence

In [ ]:
samplerate = 120
window_size = 256
window_overlap = 0.5

# Perform cross-spectral coherence analysis
cohere_stats, FP1, FP2, cohereFP = cross_spectral_coherence(data.iloc[:, 0], data.iloc[:, 1], samplerate, window_size, window_overlap)

# Print average coherence
print(f'Average Coherence (across spectrum): {cohere_stats[0]}')
print(f' Coherence (peak ts1): {cohere_stats[1]}')
print(f' Coherence (peak ts2): {cohere_stats[2]}')
print(f' Coherence (peak freq average): {cohere_stats[3]}')

# Plot the power spectral densities and coherence
plot_spectral_analysis(FP1, maxFreq=10)
plot_spectral_analysis(FP2, maxFreq=10)
plot_coherence(cohereFP, maxFreq=10)

##### Auto and Cross-Correlation

In [ ]:
# Calculate auto-correlation for each participant
lags_x, corr_x = auto_correlation(data.iloc[:, 0], 120)
plot_autocorrelation(lags_x, corr_x)

# Calculate auto-correlation for each participant
lags_x, corr_x = auto_correlation(data.iloc[:, 1], 120)
plot_autocorrelation(lags_x, corr_x)

# Calculate cross-correlation between participants
cross_loags, cross_corr = cross_correlation(data.iloc[:, 0], data.iloc[:, 1], 120)
plot_crosscorrelation(cross_loags, cross_corr)

##### Relative Phase

In [ ]:
samplerate = 120  # 100 Hz sampling rate

# Perform relative phase analysis and plot results
meanRP, sdRP, rvRP, radians = irp(data.iloc[:, 0], data.iloc[:, 1])
print(f'RP: mean = {meanRP}  sd = {sdRP}   rho = {rvRP}')
plot_irp(radians, samplerate)
plot_rp_distribution(radians)

meanRP, sdRP, rvRP, radians, peaks = drp(data.iloc[:, 0], data.iloc[:, 1], samplerate)
print(f'RP: mean = {meanRP}  sd = {sdRP}   rho = {rvRP}')
plot_drp(radians, peaks, samplerate)
plot_rp_distribution(radians, samplerate) 

#return_plot(data.iloc[:, 0], data.iloc[:, 1], samplerate)